In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype=torch.float32,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    # attn_implementation="sdpa",
    attn_implementation="flash_attention_2",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

# correct_dataset = correct_dataset[:30]
len(correct_dataset)

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
209 185 0.8851674641148325


185

In [2]:
import torch
import gc
from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from tqdm import tqdm as text_tqdm
from hidden_capacity_reasoning.utils import tokenize_single_turn

torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

for dataset_pos in tqdm(range(1, len(correct_dataset))):
    tokenized_turn = tokenize_single_turn(
        question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
        answer=correct_dataset[dataset_pos]["model_answer"],
        tokenizer=tokenizer,
    )
    break

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
tokenized_turn

In [33]:
for key in tokenized_turn.keys():
    tokenized_turn[key] = torch.tensor(tokenized_turn[key])

device = "cuda"

content_compression_mask = tokenized_turn["content_compression_mask"]

input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
# get only question part
question_input_ids = (
    tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
)
tokenizer.decode(question_input_ids[-1])

/tmp/ipykernel_21672/4083140599.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_turn[key] = torch.tensor(tokenized_turn[key])


'<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \\ldots, 99, 100 ?$\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\n'

In [ ]:
tokenizer.decode(tokenized_turn["input_ids"][content_compression_mask == 2])

'Okay, so I have this problem here: "How many numbers are in the list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it\'s something like the last term minus the first term plus one. Let me verify that.\n\nSo, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn\'t seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add something else.\n\nLet me recall the formula for the number of terms in an arithmetic sequence. I think it\'s (last term - first term) divided by the common difference plus one. Since the common difference here is 1 (because each number is one more than the previous), the formula simplifies to just (last

In [35]:
tokenizer.decode(tokenized_turn["input_ids"][content_compression_mask == 3])

'\n\nTo determine how many numbers are in the list \\(25, 26, 27, \\ldots, 99, 100\\), we can use the formula for the number of terms in an arithmetic sequence. The formula is:\n\n\\[\n\\text{Number of terms} = \\left( \\text{Last term} - \\text{First term} \\right) / \\text{Common difference} + 1\n\\]\n\nHere, the first term is 25, the last term is 100, and the common difference is 1. Plugging these values into the formula, we get:\n\n\\[\n\\text{Number of terms} = \\left( 100 - 25 \\right) / 1 + 1 = 75 + 1 = 76\n\\]\n\nThus, the number of numbers in the list is \\(\\boxed{76}\\).'

In [11]:
tokenized_turn["input_ids"][content_compression_mask == 2].shape

torch.Size([1568])

In [ ]:
tokenized_turn["input_ids"][content_compression_mask == 2]

In [ ]:
# block_size = 4
# think_block_size = tokenized_turn["input_ids"][content_compression_mask == 2].shape[0]
# block_mask = torch.arange(0, block_size).repeat(
#     think_block_size // block_size + 1,
# )
# block_mask = block_mask[:think_block_size]
#
# tokenized_turn["input_ids"][input_part_end : input_part_end + think_block_size][
#     block_mask != 0
# ] = 151662

In [17]:
tokenizer.decode([151662])

'<|fim_pad|>'

In [16]:
tokenizer.decode(tokenized_turn["input_ids"])

'<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \\ldots, 99, 100 ?$\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\nOkay, so I have this problem here: "How many numbers are in the list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it\'s something like the last term minus the first term plus one. Let me verify that.\n\nSo, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn\'t seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add something else.\n\nLet me recall the formula for the number of terms in an arithmetic sequence. I think i

In [39]:
block_size = 16
# think_block_size = tokenized_turn["input_ids"][content_compression_mask == 2].shape[0]
think_block_size = tokenized_turn["input_ids"].shape[0] - input_part_end - 1
block_mask = torch.ones(think_block_size)


original_input_ids = tokenized_turn["input_ids"].clone()
for block_pos in range(1, block_mask.shape[0] // block_size + 2):
    train_block_size = block_pos * block_size
    # temp_mask_1 = torch.ones(think_block_size)
    temp_block_mask = block_mask.clone()
    temp_block_mask[train_block_size:] = block_size

    if block_pos > 1:
        temp_block_mask[: (block_pos - 1) * block_size] = block_size

    temp_input_ids = original_input_ids.clone()
    temp_input_ids[input_part_end + 1 : input_part_end + 1 + think_block_size][
        temp_block_mask != block_size
    ] = 151662
    # print(block_mask)
    print(tokenizer.decode(temp_input_ids))
    print("==")
    print("==")
    print("==")
    # break

<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \ldots, 99, 100 ?$

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
<|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|> list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it's something like the last term minus the first term plus one. Let me verify that.

So, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn't seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add so